Funziona sia per SI che per SIR, basta scegliere il valore di mu

In [1]:
import numpy as np
import Simple_functions_SIR as SM
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import csv
import networkx as nx
import random
import pickle
import pandas as pd
import os
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')
from utils import *

In [2]:
#dataset = "pr_school";gap = 300;state_gap = 1200;nb_states_L = 3;nb_states_e = 8
#dataset = "h_school11";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 7
#dataset = "h_school13";gap = 1200;state_gap = 1200;nb_states_L = 4;nb_states_e = 29
#dataset = "InVS15"; gap = 3600; state_gap = 3600; nb_states_L = 3;nb_states_e = 12
dataset = "hypertext";gap = 1200;state_gap = 1200;nb_states_L = 3;nb_states_e = 20

In [3]:
#net = 'D'
net = 'E'
#net = 'ES'
#net = 'EST'

# Functions

In [5]:
def upload_edgelist_D(dataset,gap):

    edgelist = pd.read_csv("../../Datasets/"+dataset+".dat",sep=" ",names=["t","i","j"])
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [6]:
def upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L):
    path = '../../Generated_graphs/d2/'
    if net == 'E':
        directory = path+"%s_E_gap_%d_%d_states_e/"%(dataset,gap,nb_states_e)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'ES':       
        directory = path+"%s_ES_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])

    elif net == 'EST':       
        directory = path+"%s_EST_gap_%d_%d_states_e_%d_states_L/"%(dataset,gap,nb_states_e,nb_states_L)
        edgelist = pd.read_csv(directory+"/%d.dat"%net_idx,sep=" ",names=["t","i","j"])
    
    nb_nodes = max(max(edgelist.i),max(edgelist.j)) + 1
    nodes_list = np.arange(0,nb_nodes)
    times = [int(x/gap) for x in edgelist.t]
    edgelist.t = times

    return nodes_list,edgelist

In [ ]:
def obtain_edgelist_dict(data_):
    edgelist_dict = {}
    for t in range(max(data_.t) + 1):
        edgelist_dict[t] = []
    
    for _, row in data_.iterrows():
        if row['t'] in edgelist_dict:
            if [row['i'],row['j']] not in edgelist_dict[row['t']]:
                edgelist_dict[row['t']].append([row['i'],row['j']])
            if [row['j'],row['i']] in edgelist_dict[row['t']] and row['i'] != row['j']:
                print('error1')
        else:
            print('error2')
    return edgelist_dict

# Simulate

In [7]:
beta_range = np.logspace(-2,-1,num=10)
mu_range = np.logspace(-2,-1,num=10)

In [8]:
folder = 'results/%s_pat0_random/'%net # to save results
if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
nb_surr = 10 # nb of generated networks
nb_runs_surr = 20 # nb of simulations on each network (so 200 simulations in total)
nb_runs_D = nb_surr*nb_runs_surr

In [9]:
if net == 'D':

    # Load edgelist
    nodes_list,edgelist = upload_edgelist_D(dataset,gap)
    edgelist_dict = obtain_edgelist_dict(edgelist)
 
    non_empty_t = np.unique(edgelist.t)
    
    # Simulate
    sm = SM.Simplagion_model(edgelist_dict,nodes_list)
    for beta in beta_range:
        for mu in mu_range:
            print('beta = ',beta,'mu = ',mu)
            
            filenameR = "results/%s_pat0_random/R_%s_beta_%.3f_mu_%.3f.txt"%(net,dataset,beta,mu)
            fileR = open(filenameR, 'a+')

            for run in range(20):
                t0 = random.choice(non_empty_t)
                pat0 = random.choice(nodes_list)
                sm.initialize_time0(pat0)
                R0,Rinf = sm.simulate(beta,mu,t0) # obtain R0 and Rinf
                fileR.write('%f %f\n'%(R0,Rinf))
            fileR.close()

In [11]:
if net != 'D':
    for net_idx in tqdm(range(nb_surr)): 
        print('net_idx',net_idx)
        # Load edgelist
        nodes_list,edgelist = upload_edgelist_surrogate(dataset,net,net_idx,nb_states_e,gap,nb_states_L)
        edgelist_dict = obtain_edgelist_dict(edgelist)

        non_empty_t = np.unique(edgelist.t)

        # Simulate
        sm = SM.Simplagion_model(edgelist_dict,nodes_list)
        for beta in beta_range:
            for mu in mu_range:
                print('beta = ',beta,'mu = ',mu)

                filenameR = "results/%s_pat0_random/R_%s_beta_%.3f_mu_%.3f.txt"%(net,dataset,beta,mu)
                fileR = open(filenameR, 'a+')

                for run in range(nb_runs_surr):
                    t0 = random.choice(non_empty_t)
                    pat0 = random.choice(nodes_list)
                    sm.initialize_time0(pat0)
                    R0,Rinf = sm.simulate(beta,mu,t0) # obtain R0 and Rinf
                    fileR.write('%f %f\n'%(R0,Rinf))
                fileR.close()

  0%|          | 0/10 [00:00<?, ?it/s]

net_idx 0
beta =  0.02 mu =  0.01
net_idx 1
beta =  0.02 mu =  0.01
net_idx 2
beta =  0.02 mu =  0.01
net_idx 3
beta =  0.02 mu =  0.01
net_idx 4
beta =  0.02 mu =  0.01
net_idx 5
beta =  0.02 mu =  0.01
net_idx 6
beta =  0.02 mu =  0.01
net_idx 7
beta =  0.02 mu =  0.01
net_idx 8
beta =  0.02 mu =  0.01
net_idx 9
beta =  0.02 mu =  0.01


KeyboardInterrupt: 

In [ ]:
#####################